### Import of modules/libs

In [1]:
import collada as cl
import pandas as pd
import numpy as np


### Import of data

In [2]:

pathToFile :str = 'cubes_scene.dae'
with open(file=pathToFile, mode="rb") as cubesSceneFile:
    mesh = cl.Collada(cubesSceneFile, ignore=[cl.common.DaeUnsupportedError, cl.common.DaeBrokenRefError])
    mesh.assetInfo.upaxis= "Z_UP"
    print("This is the whole scene source list object \t",mesh)
    # print(mesh.geometries)
    print("This is the first geometry object \t\t",mesh.geometries[0])
    

This is the whole scene source list object 	 <Collada geometries=130>
This is the first geometry object 		 <Geometry id=Cube_130-mesh, 1 primitives>


### Filtering by colour

In [3]:
yellowMaterial = 'Material.001'
blueMaterial = 'Material.002'

listOf_yellows :list = []
listOf_blues :list = []
listOf_None :list = []
# It is possible to filter out cubes via material. 001 is BLUE, 002 is yellow

sceneGraphNode = mesh.scene.nodes[0]
listOf_sourceGeoNodes = list(mesh.scene.objects('geometry'))
print("Amount of objects(geometry) in scene _> \t",len(listOf_sourceGeoNodes))
listOf_GeoPrims = list(listOf_sourceGeoNodes[0].primitives())
print("Amount of geometry in objects(geometry) _> \t",len(listOf_GeoPrims))
listOf_triangles = listOf_GeoPrims[0]
print("Amount of TRIANGLES in geometry _> \t",len(listOf_triangles))

for idx, obj in enumerate(listOf_sourceGeoNodes):
    # print(idx)
    geo = list(obj.primitives())
    triangle = geo[0]
    # print((triangle))
    # typeOfMaterial :str = lambda x : ((obj[i].primitives())[0]).material.name
    #? There are None types for some reason
    #? Cubes without material have diffrent index in blender and diffrent here
    if triangle.material is None:
        # print(type(triangle.material))
        
        listOf_None.append(obj)
        continue
    
    # print(triangle.material.name)
    if (triangle.material.name == yellowMaterial):
        listOf_yellows.append(obj)
    else:
        listOf_blues.append(obj)
print("\nThis is the amount of yellows",  len(listOf_yellows))
print("This is the amount of blues",    len(listOf_blues))
print("This is the amount of Nones",    len(listOf_None))

Amount of objects(geometry) in scene _> 	 130
Amount of geometry in objects(geometry) _> 	 1
Amount of TRIANGLES in geometry _> 	 12

This is the amount of yellows 107
This is the amount of blues 19
This is the amount of Nones 4


### Filtering by area xyz (0;15)

In [4]:
#? plan:
#? go though array of obj;
#? go though array of verts of every single component and see if np.any of them returns false
#? if true then append?
#? only ONE vert needs to be in area to consider a obj being in area

lower_bound , upper_bound = 0, 15
listOf_inArea :list = []
listOf_outOfBounds :list = []
i:int = 10
for idx_1, obj in enumerate(listOf_sourceGeoNodes):
    geo = list(obj.primitives())
    triangle = geo[0]
    
    for  idx_2 , arrayOfVerts in enumerate(triangle.vertex):
        # numpy accepts only bitwise operators here
        # ifInArea = np.all((arrayOfVerts > 0) & (arrayOfVerts < 15))
        # x,y,z = arrayOfVerts[0], arrayOfVerts[1], arrayOfVerts[2]
        # if (0<x) and (x<15):
        #     ifInArea_x = True
        # else:
        #     ifInArea_x = False
            
        # print(arrayOfVerts)
        # print(vertInArea)
        
        vertInArea = np.all((lower_bound < arrayOfVerts) & (arrayOfVerts < upper_bound)) 
        if (vertInArea == True):
            listOf_inArea.append(obj)
            break
        if (idx_2 == len(triangle.vertex) -1 ):
            listOf_outOfBounds.append(obj)
    
print("This is the amount of inArea",           len(listOf_inArea))
print("This is the amount of outOfBounds",      len(listOf_outOfBounds))

This is the amount of inArea 3
This is the amount of outOfBounds 127


### Transforming the scene

In [5]:
quickList = [[0,0,0,0],   [0,0,0,100],    [0,0,0,0],    [0,0,0,0]]
offsetForMatrix = np.array( quickList,dtype= "float16")
print(len(offsetForMatrix))
transforms_old = (mesh.scene.nodes[0]).transforms[0]
transforms_old = transforms_old.matrix

print("This is the old matrix\n",transforms_old)
print("This is the new matrix\n",transforms_old + offsetForMatrix)

4
This is the old matrix
 [[ -0.2502034   -0.965825    -0.06767803 -13.11428   ]
 [  0.8722231   -0.1945087   -0.4487686  -22.1053    ]
 [  0.420268    -0.1713138    0.8910816  -46.21673   ]
 [  0.           0.           0.           1.        ]]
This is the new matrix
 [[-2.502034e-01 -9.658250e-01 -6.767803e-02 -1.311428e+01]
 [ 8.722231e-01 -1.945087e-01 -4.487686e-01  7.789470e+01]
 [ 4.202680e-01 -1.713138e-01  8.910816e-01 -4.621673e+01]
 [ 0.000000e+00  0.000000e+00  0.000000e+00  1.000000e+00]]


## Creating new objects

In [6]:
newSceneMesh = cl.Collada()


# sceneGraphNode.transforms[0] = cl.scene.MatrixTransform(offsetForMatrix)
# print(mesh.geometries[0])
# print(listOf_yellows[0])

In [7]:
testObj = (listOf_yellows[0])
testObj = testObj.original

In [8]:
newSceneMesh.effects = mesh.effects
newSceneMesh.materials = mesh.materials


# matnode = cl.scene.MaterialNode("materialref", mesh.materials, inputs=[])
# geomnode = cl.scene.GeometryNode(geom, [matnode])
# node = cl.scene.Node("node0")
# myscene = cl.scene.Scene("myscene", [node])
# mesh.scenes.append(myscene)
newSceneMesh.scene
# newSceneMesh.scene.nodes = mesh.scene.nodes

mesh.geometries = []
for obj in listOf_yellows:
    newSceneMesh.geometries.append(obj.original)
# cool = ((newSceneMesh.scene.node).transforms[0])
# print(cool.transforms[0])
print("OLD _>")
print(mesh)
print(mesh.nodes)
print(mesh.materials)
print(mesh.effects)
print(mesh.geometries)

print("NEW _>")
print(newSceneMesh)
print(newSceneMesh.nodes)
print(newSceneMesh.materials)
print(newSceneMesh.effects)
print(newSceneMesh.geometries)



OLD _>
<Collada geometries=0>
[]
[<Material id=Material_001-material effect=Material_001-effect>, <Material id=Material_002-material effect=Material_002-effect>]
[<Effect id=Material_001-effect type=lambert>, <Effect id=Material_002-effect type=lambert>]
[]
NEW _>
<Collada geometries=107>
[]
[<Material id=Material_001-material effect=Material_001-effect>, <Material id=Material_002-material effect=Material_002-effect>]
[<Effect id=Material_001-effect type=lambert>, <Effect id=Material_002-effect type=lambert>]
[<Geometry id=Cube_130-mesh, 1 primitives>, <Geometry id=Cube_129-mesh, 1 primitives>, <Geometry id=Cube_127-mesh, 1 primitives>, <Geometry id=Cube_125-mesh, 1 primitives>, <Geometry id=Cube_124-mesh, 1 primitives>, <Geometry id=Cube_122-mesh, 1 primitives>, <Geometry id=Cube_121-mesh, 1 primitives>, <Geometry id=Cube_120-mesh, 1 primitives>, <Geometry id=Cube_119-mesh, 1 primitives>, <Geometry id=Cube_118-mesh, 1 primitives>, <Geometry id=Cube_117-mesh, 1 primitives>, <Geometry i

In [9]:
newSceneMesh.write("test.dae")